In [50]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
import src.dataset as ds
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
WORDS_VECTORS_DIR = 'word_vectors/'
LYRICS_DIR = 'Data/'
GLOVE_DIR = os.path.join(WORDS_VECTORS_DIR, 'glove.6B')
TEXT_DATA = os.path.join(LYRICS_DIR, 'unified_lyrics_dump.txt')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 200000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# Show Word Embeddings

In [54]:
print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'), 'rb') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

Indexing word vectors.


In [55]:
with open(os.path.join(WORDS_VECTORS_DIR, 'glove_embeddings.pickle'), 'wb') as f:
    pickle.dump(embeddings_index, f)

In [58]:
with open(os.path.join(WORDS_VECTORS_DIR, 'glove_embeddings.pickle'), 'rb') as f:
    s = pickle.load(f)

In [ ]:
def check_our_corpus(our_words):
    not_found = []
    for word in our_words:
        if word.encode() not in embeddings_index:    
            not_found.append(word)
    return not_found

In [ ]:
vocab = ds.load_vocab()
not_found = check_our_corpus(vocab)

In [38]:
with open(TEXT_DATA, 'r') as f:
    text = [f.read()]

In [39]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)